In [ ]:
import torch
import numpy as np
import os
import json
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, PreTrainedTokenizer

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [ ]:
# llm utility codes

# generate llm text without watermarking
def unwatermarked_token_generation(probs, counter, vocab_size):
    gen_tokens = torch.multinomial(probs, 1)
    return gen_tokens

#############
# GUMBEL Watermarking

# generate llm text with gumbel watermarking
def gumbel_token_generation(probs: torch.Tensor, counter, vocab_size, seed=1234):
    device = probs.device
    g = torch.Generator()
    g.manual_seed(seed + counter)
    unif_noise = torch.rand(vocab_size, generator=g).to(device)
    gumbel_ratio = torch.log(unif_noise) / probs[0]
    return torch.argmax(gumbel_ratio).view(-1, 1)


def pivot_statistic_gumbel_func(gen_tokens, vocab_size, seed=1234):
    # gen_tokens is a numpy array, so convert into torch Tensor for torch operations
    pivot_stat = []
    for counter, gen_token in enumerate(gen_tokens):
        g = torch.Generator()
        g.manual_seed(seed + counter)
        unif_noise = torch.rand(vocab_size, generator=g)
        pivot_stat.append(-torch.log(1 - unif_noise[gen_token]).item())
    return pivot_stat


######################
# Inverse Watermarking

# generate llm text with inverse watermarking
def inverse_token_generation(probs: torch.Tensor, counter, vocab_size, seed=1234):
    g = torch.Generator()
    g.manual_seed(seed + counter)
    unif_noise = torch.rand(1, generator=g)  # (1,)
    pi = torch.randperm(vocab_size, generator=g)  # random permutation (vocab_size, )
    inv_pi = torch.empty_like(pi)
    inv_pi[pi] = torch.arange(vocab_size)

    probs_shuffled = probs[0, inv_pi]  # probs is shape (1, vocab_size)
    cdf = torch.cumsum(probs_shuffled, dim=0)  # (vocab_size,)
    index = torch.searchsorted(
        cdf, unif_noise.item(), right=False
    )  # Find the first index where cdf exceeds unif_noise

    # Return the original vocab index corresponding to the sampled one
    return inv_pi[index].view(-1, 1)


def pivot_statistic_inverse_func(gen_tokens, vocab_size, seed=1234):
    pivot_stat = []
    for counter, gen_token in enumerate(gen_tokens):
        g = torch.Generator()
        g.manual_seed(seed + counter)
        unif_noise = torch.rand(1, generator=g)  # (1,)
        pi = torch.randperm(vocab_size, generator=g)  # random permutation (vocab_size, )
        normalized = pi[gen_token] / (vocab_size - 1) # as pi[gen_token] yields a value between 0 to (vocab_size - 1)
        pivot_stat.append(1 - np.abs((normalized - unif_noise).item()))  # 1 - <..> so that under H0, mean is small
    return pivot_stat


# Common function that can be used to generate tokens using LLM
def generate_llm_tokens(
    prompt: str,
    tokenizer: PreTrainedTokenizer,  # usually AutoTokenizer
    model,  # usually AutoModelForCausalLM
    token_generation_func,  # a token generation function, or a dict <start_index>:<token_gen_func>, see below.
    verbose=False,
    prompt_tokens=50,  # take the first 50 tokens of prompt as input
    out_tokens=50,  # output next 50 tokens
    vocab_size=None,
):
    # It is also possible to provide input to the token_generation_func a dictionary of the following form
    # {
    #     "0": watermark_func_1,
    #     "t1": watermark_func_2,
    #     "t2": watermark_func_3,
    #     ...
    # }
    # It allows to use different watermarking scheme to be added in between
    if vocab_size is None or vocab_size < 0:
      vocab_size = model.get_output_embeddings().weight.shape[0]

    tokens = tokenizer.encode(
        prompt, return_tensors="pt", truncation=True, max_length=2000
    )  # get the token vector
    torch_prompt = tokens[:, :prompt_tokens]  # give the first prompt_tokens as input
    inputs = torch_prompt.to(model.device)
    counter_range = tqdm(range(out_tokens)) if verbose else range(out_tokens)
    gen_tokens = []
    for counter in counter_range:
        with torch.no_grad():
            output = model(inputs)  # apply the model
        probs = torch.nn.functional.softmax(
            output.logits[:, -1, :], dim=-1
        )  # apply softmax over the last dimension

        # extract the token generation function
        if isinstance(token_generation_func, dict):
            token_change_times = [int(x) for x in list(token_generation_func.keys())]
            for key in sorted(token_change_times, reverse=True):
                if key <= counter:
                    token_gen_func = token_generation_func[str(key)]
                    break
        else:
            token_change_times = []  # no change times
            token_gen_func = token_generation_func

        gen_token = token_gen_func(
            probs, counter + prompt_tokens, vocab_size
        )  # calculate the token

        gen_token = gen_token.to(model.device)  # move to the device
        gen_tokens.append(gen_token.item())  # append to gen_tokens as numpy array
        inputs = torch.concat(
            (inputs, gen_token), dim=1
        )  # first dim = batch_size, second dim needs to merge

    # at the end, produce the decoded text
    out_text = tokenizer.decode(inputs[0])
    return {
        "prompt": tokenizer.decode(torch_prompt[0]),
        "gen_tokens": gen_tokens,
        "output": out_text
    }

In [4]:
root_data_path = "../data"
output_data_path = "../data/output"

def get_prompts():
    with open(os.path.join(root_data_path, "prompts_subset.txt"), "r", errors="ignore") as f:
        prompts = f.read().split("\n===\n")
    return prompts

prompt_list = get_prompts()
print(prompt_list[0])

The Mapes family of Effingham enjoy the Lincoln Park Zoo in Chicago with their children including their adopted children, Regino and Regina, who were born in the Philippines.
Misty Mapes and her husband, Patrick, of Effingham always had a desire to add to their family through adoption.
That dream became a reality in part due to Gift of Adoption Fund, a nonprofit organization that provides financial support to families that need help to pay for the hefty cost of adopting a child.
The Mapes, who have two biological children, Braydon, 16, and Madison, 11, were able to adopt 8-year-old twins, Regina and Regino — who go by Ina and Ino — about a year ago from the Philippines. They received a $4,000 grant that helped them pay travel expenses to the Philippines to bring the children home.
Though they had always had tossed around the idea of adoption, they were spurred to take action when their older son asked them about it a few years ago.
"He said, 'Hey. Can we adopt? I'd like to have a broth

In [5]:
model_name = "facebook/opt-125m"

device = torch.device("cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

vocab_size = model.get_output_embeddings().weight.shape[0]
print(f"There are {vocab_size} many words in vocabulary")
print(f"The model {model_name} is loaded on device: {device}")

torch.set_num_threads(8) # set only 8 threads to use

There are 50272 many words in vocabulary
The model facebook/opt-125m is loaded on device: cpu


In [8]:
# the data configuration to run for
data_configuration = {
        "fname": "data_inverse_n500.json",
        "prompt_tokens": 50,
        "out_tokens": 500,
        "token_generation_func": {
            "0": unwatermarked_token_generation,
            "100": inverse_token_generation,
            "200": unwatermarked_token_generation,
            "400": inverse_token_generation,
            "450": unwatermarked_token_generation,
        },
        "pivot": pivot_statistic_inverse_func
    }

In [9]:
# Run the main simulation loop
prompt_tokens: int = data_configuration.get("prompt_tokens", 0)
out_tokens: int = data_configuration.get("out_tokens", 0)
pivot_func = data_configuration.get("pivot")
pivot_seed = 1234 + prompt_tokens  # this is where the seed for pivot statistic will start from    
token_generation_func_serialized = {
    k: v.__name__ for k, v in data_configuration.get("token_generation_func", {}).items()
}
    
data_list = []
save_every = 10
counter = 0
for prompt in tqdm(prompt_list):
    counter += 1
    response = generate_llm_tokens(
        prompt,
        tokenizer,
        model,
        token_generation_func=data_configuration.get("token_generation_func", {}),
        verbose=False,
        out_tokens=out_tokens,
        prompt_tokens=prompt_tokens
    )
    if pivot_func is not None:
        # calculate pivot function as well
        gen_tokens = response["gen_tokens"]
        response["pivots"] = pivot_func(gen_tokens, seed = pivot_seed, vocab_size = vocab_size)
    data_list.append(response)

    if counter % save_every == 0:
        # save the JSON
        data_outfile = data_configuration.get("fname", "data.json")
        with open(os.path.join(output_data_path, data_outfile), "w") as f:
            json.dump({
                "configuration": {
                    "token_generation_func": token_generation_func_serialized,
                    "model_name": model_name,
                    "prompt_tokens": prompt_tokens,
                    "out_tokens": out_tokens,
                    "vocab_size": vocab_size
                },
                "data": data_list
            }, f)
            f.close()


# save the JSON at last as well
data_outfile = data_configuration.get("fname", "data.json")
with open(os.path.join(output_data_path, data_outfile), "w") as f:
    json.dump({
        "configuration": {
            "token_generation_func": token_generation_func_serialized,
            "model_name": model_name,
            "prompt_tokens": prompt_tokens,
            "out_tokens": out_tokens,
            "vocab_size": vocab_size
        },
        "data": data_list
    }, f)
    f.close()


  0%|          | 0/200 [00:00<?, ?it/s]